# The inorganic carbon balance and fate of anthropogenic carbon in a temperate fjord system

We are trying to write a paper investigating the fate of anthropogenic carbon in a fjord system, using the SalishSeaCast Model, with custom-built carbonate chemistry module, as our tool. The basic idea is to run the SKØG model in 3 configurations - present-day co2 forcing; preindustrial co2 forcing + present-day boundary conditions; and preindustrial co2 forcing + preindustrial boundary conditions. 

Then we try to see: 

a) where the extra carbon is 

b) how it gets transported thru lateral (b1) and air-sea (b2) boundary and how that balance  changes in the 3 scenarios.

c) the effect this has on ecologically-meaningful quantities (eg $\Omega_A$, pH)  

This notebook tries to keep track of where the model code is, the run files, the results, and all the different bits of analysis are for a) Tereza, who could stand to be more organized, and b) advisors/collaborators - Debby, Susan, et al. The idea is to put links to relatively tidy notebooks that distill what has been done. The paper is being written on Overleaf.   

Outline of doc:

* **1a** Model Code/ Config
* **1b** Init atmospheric CO2
* **1c** PI boundary conditions
* **2** Model Runs Description
* **3** Model Evaluations
* **4** Results

QQ means to fill in
TD means... todo

### 1a) Model Code/ Configuration:
    
    Changeset:
    changeset:   1702:7bb2e93c05d8
    Main code of SKOG is in p4zcar.F90 and p4zflx.F90,
    and we couple the mocsy system of subroutines. 
    Equations from Moore-Maley 2016
    

### 1b) Figuring out how to initialize atmospheric CO2 

We have seasonally varying CO2. We have found a sum-of-gaussians expression for CO2 based on means of empirical observations by NOAA at La Jolla and Pt Barrow. 

The notebook in which I did this & set the parameters of the gaussians is [here](https://nbviewer.jupyter.org/urls/bitbucket.org/tjarnikova/analysis-tereza/raw/923f9523c331f150164d5929e3019084a21a853c/notebooks/carbon_dev/MOCSY_and_FLUX/CO2_obs_brokenscipy.ipynb).


The final code is here, from p4zflx: 
    
          zz_yearcyc =  zz_amp * exp( -((zz_day - zz_ctr)/zz_wid)**2)&
              + zz_amp2 * exp( -((zz_day - zz_ctr2)/zz_wid2)**2)&
              + zz_amp3 * exp( -((zz_day - zz_ctr3)/zz_wid3)**2)

      zz_atcco2 = (zz_year+(zz_day/365))*zZ_LRslop+zz_LRint + zz_yearcyc
      
The zz_amp, zz_wid, zz_ctr get set in namelist_smelt_skog which is in, eg: 

    SS-run-sets/SalishSea/nemo3.6/tereza/BASERUN_PREIND_EXP/PI_ACBC_1st2015_nolatflux
      

### 1c) Figuring out the (bloody) preindustrial boundary conditions

We thought about several ways to set the preindustrial boundary conditions. This is a 2 part process - figuring out the age of the water and then figuring out the relationship between age and CO2 levels (what I call CO2 intrusion - extra CO2 that a water parcel sees due to anthropogic carbon increase. 

###### For age: We looked at various options:
>* Tetjana Ross's isopycnal vs local ventilation
>* Making a convoluted argument about watermass identity & age based partly on Thomson/Krassowski paper
>* Andrew Shao's suggestion: use observed CFC-11 freon age by isopycnal to get an analytic equation for age.

We ended up using the last one: freon-11 observations from between -135 and - 125 to set an analytic equation. Andrew's data is in the csv P01_with_age.csv. A notebook that illustrates how we got that equation is [here](https://nbviewer.jupyter.org/urls/bitbucket.org/tjarnikova/analysis-tereza/raw/923f9523c331f150164d5929e3019084a21a853c/notebooks/carbon_dev/PI_CARBON_PAPER/PI_BOUND_COND/CLEAN/Andrew_age_cfc_to_pco2_conv_to_new_DIC_ncs.ipynb).


###### For co2 intrusion:

> * Feely et al method of 2 different relationships pivoting at isopycnal 25 (no good, creates a discontinuity)

> * Ana Franco's inferred nDIC increase from observations

> * simple pco2 intrusion difference

> * modified Gruber 1996 methodology - pco2 intrusion scales with age, considers AOU and CO2 disequilibrium when last at surface.

Used the last one. Method summary:

Uses a derived relationship between potential density (sigma0) and cfc-freon11 estimated watermass age to determine what atmospheric co2 the watermass saw when it was last at surface. Calculates AOU at in-situ pressure using the gsw toolbox. Subtracts AOU from present-day DIC to
get preformed DIC, the assumption being that the AOU accounts for alterations to DIC due to biological activity that have happened since the watermass was at surface, with a ratio of AOU:DIC
of C:O (117:170). Uses this preformed DIC and present-day T,S,TA to calculate preformed pco2 at surface. Then uses
atmospheric co2 when the watermass was last at surface to calculate the pco2 disequilibrium. Then uses preformed_pco2 + pco2_disequilibrium to calculate preindustrial preformed DIC. Then it substracts preindustrial preformed DIC from present day preformed DIC to get delta DIC. It then
subtracts delta DIC from present day DIC to get the final preindustrial DIC concentration.

Practically I did this in 2 steps: 
> * First I did the above calculation and saved an .nc with the results of it and the intermediate steps. The code for that is (for the JS boundary) is [here](https://nbviewer.jupyter.org/urls/bitbucket.org/tjarnikova/analysis-tereza/raw/923f9523c331f150164d5929e3019084a21a853c/notebooks/carbon_dev/PI_CARBON_PAPER/PI_BOUND_COND/CLEAN/create_bcs_JS_AOU_meth.ipynb). 
> * Then I copied the preind DIC to ncs of the form PREIND_BC_v201905_y2015m06d30.nc for use in the model. NB [here](https://nbviewer.jupyter.org/urls/bitbucket.org/tjarnikova/analysis-tereza/raw/923f9523c331f150164d5929e3019084a21a853c/notebooks/carbon_dev/PI_CARBON_PAPER/PI_BOUND_COND/CLEAN/Conv_to_new_DIC_ncs.ipynb)

The nc's themselves are here: 
    
    tracers/west/PREIND_BCCOND/PREIND_BC_v201905/PREIND_BC_v201905_y2015m06d30.nc

referenced in namelist_top_TracerDefAndBdy 
in the rundir: 
    
    SS-run-sets/SalishSea/nemo3.6/tereza/BASERUN_PREIND_EXP/PI_ACBC_1st2015_nolatflux



### 2) Model Runs:

3 runs
> main run directory is: /SS-run-sets/SalishSea/nemo3.6/tereza/BASERUN_PREIND_EXP/

> main results directory (on salish) is: /data/tjarniko/results/BASERUN_EXP

* BR: present-day atmo CO2 with present-day boundary conditions
    
> yamls/forcings: BR_1st2015, BR_2nd2015  

> results: MAIN/BR_2nd_2015 MAIN/BR_2016

> results of spinup: NOT_MAIN_ANALYSIS/BR_1st_2015
    
* PI_AC: preindustrial atmo CO2 with present-day BC

> yamls/forcings: PREIND_2015_year3

> results: MAIN/PI_3rd_2015/ MAIN/PI_2016

> results of spinup: NOT_MAIN_ANALYSIS/: PI_1st_2015/ PI_2nd_2015/

* PI_ACBC: preindustrial atmo CO2 with preindustrial BC

> yamls/forcings: PI_ACBC_1st2015_nolatflux

> results: MAIN/PI_ACBC_2015/

> results of spinup: MAIN/PI_ACBC_2015_2/


### 3) Checking Spinup

We used the somewhat informal metric of "is the difference of surface DIC, nitrate, and salinity between the spinup and non-spinup year tending sufficiently to a steady state around zero?" For the BR run, this took 1 year, for PI_AC, 2. PI_ACBC started at the end of the PI_AC model year and seems to only take one year. 

PI_ACBC is [here](https://nbviewer.jupyter.org/urls/bitbucket.org/tjarnikova/analysis-tereza/raw/2ef59aeb166d9a1b5dc15bb1ae7dc407a1bc3fb4/notebooks/carbon_dev/PI_CARBON_PAPER/MAIN_ANALYSIS/CLEAN/KEY_SPINUP/SPINUP_PIACBC_surface_metrics_DIC_NIT_SAL.ipynb)

PI_AC and BR is [here](https://nbviewer.jupyter.org/urls/bitbucket.org/tjarnikova/analysis-tereza/raw/2ef59aeb166d9a1b5dc15bb1ae7dc407a1bc3fb4/notebooks/carbon_dev/PI_CARBON_PAPER/MAIN_ANALYSIS/CLEAN/KEY_SPINUP/SPINUP_PIAC_BR_surface_metrics_DIC_NIT_SAL.ipynb)

### 4) Model Evaluation:

Did only a very standard model evaluation (bias, RMSE, Willmott) for [BR2015](https://nbviewer.jupyter.org/urls/bitbucket.org/tjarnikova/analysis-tereza/raw/02cff46226d00a23f5701ef176bb81512086caf5/notebooks/carbon_dev/PI_CARBON_PAPER/MAIN_ANALYSIS/CLEAN/KEY_MODEVAL/MODEL_EVAL_BR2015.ipynb) and [BR2016](https://nbviewer.jupyter.org/urls/bitbucket.org/tjarnikova/analysis-tereza/raw/c9becba2989b111cc414d9e801ff4e740db380db/notebooks/carbon_dev/PI_CARBON_PAPER/MAIN_ANALYSIS/CLEAN/KEY_MODEVAL/MODEL_EVAL_BR2016.ipynb) against the Ianson et al GRL 2016 paper. Metrics look pretty good (high 0.8s or low 0.9s for both DIC and TA for both BR_2015 and BR_2016 years.)  

    DIC 2015
    metrics calculated on FULL DATASET
    bias =  -7.941430016773211
    RMSE = 91.32960211615475
    Willmott = 0.9080898747900045

    DIC 2016
    metrics calculated on FULL DATASET
    bias =  -15.04726497771071
    RMSE = 79.8218111638049
    Willmott = 0.9343403526309809

    TA 2015
    metrics calculated on FULL DATASET
    bias =  24.789328959731847
    RMSE = 91.78920645552569
    Willmott = 0.8508141389853379

    TA 2016
    metrics calculated on FULL DATASET
    bias =  25.166037944106847
    RMSE = 80.5158220353599
    Willmott = 0.8898795425494045


### 5) Results:

#### 5a) Mass Balances
   
 Still-too-large notebook [here](https://nbviewer.jupyter.org/urls/bitbucket.org/tjarnikova/analysis-tereza/raw/2ef59aeb166d9a1b5dc15bb1ae7dc407a1bc3fb4/notebooks/carbon_dev/PI_CARBON_PAPER/MAIN_ANALYSIS/CLEAN/MASSBAL_FATE_anthropogenic_carbon_PI_BR_2015.ipynb)
 
    TD: Needs reworking to be more concise; to include PI_ACBC 

#### 5b1) Lateral Fluxes

See notebook above. 

    TD: Needs reworking to be more concise; to include PI_ACBC 


#### 5b2) Air-Sea Fluxes

 CO2 vs windspeed and representative days [here](https://nbviewer.jupyter.org/urls/bitbucket.org/tjarnikova/analysis-tereza/raw/2ef59aeb166d9a1b5dc15bb1ae7dc407a1bc3fb4/notebooks/carbon_dev/PI_CARBON_PAPER/MAIN_ANALYSIS/CLEAN/KEY_AIRSEAFLUX/AIRSEAFLUX_highlights.ipynb)
 
 Monthly analysis of PI-BR differences [here](https://nbviewer.jupyter.org/urls/bitbucket.org/tjarnikova/analysis-tereza/raw/2ef59aeb166d9a1b5dc15bb1ae7dc407a1bc3fb4/notebooks/carbon_dev/PI_CARBON_PAPER/MAIN_ANALYSIS/CLEAN/KEY_RESVIZ/MONTHLY_AVG_CO2_surface_flux.ipynb)
    
    QQ - why is this difference almost uniform?
 
 

#### 5c) Ecologically Relevant Quanitities and horizon shoaling and things

    What are the saturation horizons?
        Present day values:  
        -Jdf: ~45m summer / 90m winter  
        -Nsog ~30m s / ~20m w  
        -Csog ~40m s / ~25m w  
        -Haro 60m s / 65m w
    
    Where do they shoal the most?
        The inner strait (~10m summer, 20-30 m winter)  
        vs Haro/Jdf (<5 m both summer & winter)
        
   Notebook containing analysis is [here](https://nbviewer.jupyter.org/urls/bitbucket.org/tjarnikova/analysis-tereza/raw/tip/notebooks/carbon_dev/PI_CARBON_PAPER/MAIN_ANALYSIS/CLEAN/KEY_RESVIZ//OmA_horizon_and_shoaling_maps_and_regional_avgs_using_2algorithms_shallowdeep.ipynb) 

#### 5d) Where is the Strait most sensitive to intrusion?

    Where? Inner Strait seems more sensitive than JdF 
    
   [here](https://nbviewer.jupyter.org/urls/bitbucket.org/tjarnikova/analysis-tereza/raw/tip/notebooks/carbon_dev/PI_CARBON_PAPER/MAIN_ANALYSIS/CLEAN/KEY_RESVIZ//Monthly_avg_JAN_JUL_PIBR_oma_Sensitivity.ipynb)